In [1]:
import pandas as pd
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [2]:
df = df[df['Borough']!="Not assigned"]
df[df['Neighbourhood']=='Not assigned']
dfconcat = df.groupby(["Postcode","Borough"], as_index = False).agg(lambda x: ", ".join(sorted(set(x))))
dfconcat
dfconcat.shape

(103, 3)

In [17]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f93e94cd38494540be46172b8403264a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='e8e3YUD8T2RhefBajw9jSjSIIcGsD0HjAYkVPnTL0lVg',
    ibm_auth_endpoint="https://iam.au-syd.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_f93e94cd38494540be46172b8403264a.get_object(Bucket='capstoneweek3-donotdelete-pr-5r9pzhllyb7nfs',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_merged = pd.merge(dfconcat, df_data_0, left_on='Postcode', right_on = 'Postal Code')
df_merged.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [20]:
df_Toronto = df_merged[df_merged['Neighbourhood'].str.contains('Toronto')]
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",M3K,43.737473,-79.464763
40,M4J,East York,East Toronto,M4J,43.685347,-79.338106
46,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M5J,43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576


In [21]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

### Use geopy library to get the latitude and longitude values of TORONTO

In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


### Create map of TORONTO with neighborhoods superimposed on top

In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'X5XYTV4ABDYILZ2CWBIHI13WDWZ3XWJKCMWRIRBXQIOBRESY' # your Foursquare ID
CLIENT_SECRET = '3SWJLCKFFHGFACAHCKD3SFRYSAUU1HUKZSW2SBGSYWJ1Z022' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' , CLIENT_ID)
print('CLIENT_SECRET:' , CLIENT_SECRET)

Your credentails:
CLIENT_ID:  X5XYTV4ABDYILZ2CWBIHI13WDWZ3XWJKCMWRIRBXQIOBRESY
CLIENT_SECRET: 3SWJLCKFFHGFACAHCKD3SFRYSAUU1HUKZSW2SBGSYWJ1Z022


In [33]:
df_Toronto.reset_index(drop = True, inplace = True)

In [34]:
df_Toronto

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3K,North York,"CFB Toronto, Downsview East",M3K,43.737473,-79.464763
1,M4J,East York,East Toronto,M4J,43.685347,-79.338106
2,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
3,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M5J,43.640816,-79.381752
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576
5,M5S,Downtown Toronto,"Harbord, University of Toronto",M5S,43.662696,-79.400049
6,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",M8V,43.605647,-79.501321


### Exploring first neighborhood in our dataframe

In [35]:
df_Toronto.loc[0,'Neighbourhood']

'CFB Toronto, Downsview East'

In [37]:
neighbourhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_Toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of CFB Toronto, Downsview East are 43.737473200000004, -79.46476329999999.


### Now, getting top 100 venues in CFB Toronto, Downsview East within a radius of 500 meters.

In [38]:
# type your answer here
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6cab271e152c001b6a7315'},
 'response': {'venues': [{'id': '4bcb7c0e511f95214b99b1c7',
    'name': 'Buzz Buzz Pizza',
    'location': {'lat': 43.730591086427815,
     'lng': -79.46696345586525,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.730591086427815,
       'lng': -79.46696345586525}],
     'distance': 786,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1584180063',
    'hasPerk': False},
   {'id': '4e8d0d91d5fbbbb6b2f31bac',
    'name': 'Ancaster Park',
    'location': {'address': 'North York',
     'crossStreet': 'Ancaster Rd. & Plewes Rd.',


### Clean json and creating a pandas dataframe

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
#venues = results['response']['groups'][0]['items']
venues = results['response']['venues']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng'] #changed this code from original one
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,categories
0,Buzz Buzz Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",43.730591,-79.466963,Pizza Place
1,Ancaster Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.734706,-79.464777,Park
2,Sheppard West Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",43.749503,-79.462276,Metro Station
3,St. Norbert Catholic School,"[{'id': '4f4533804b9074f6e4fb0105', 'name': 'E...",43.734756,-79.461391,Elementary School
4,st. nobeet church,[],43.731991,-79.465381,None


In [42]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

71 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names=df_Toronto['Neighbourhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )



CFB Toronto, Downsview East
East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Harbord, University of Toronto
Humber Bay Shores, Mimico South, New Toronto


In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto, Downsview East",43.737473,-79.464763,Canada's Gardenland,43.737461,-79.466000,Construction & Landscaping
2,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
3,East Toronto,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park
4,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park


In [47]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CFB Toronto, Downsview East",3,3,3,3,3,3
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,4,4,4,4,4,4
"Harbord, University of Toronto",35,35,35,35,35,35
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Humber Bay Shores, Mimico South, New Toronto",11,11,11,11,11,11
North Toronto West,21,21,21,21,21,21


In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 99 uniques categories.


### Analyzing Each Neighborhood

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant,Flower Shop,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Travel,Gift Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Liquor Store,Lounge,Metro Station,Mexican Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Pub,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto, Downsview East",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,East Toronto,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,East Toronto,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_onehot.shape

(274, 99)

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(7, 99)

### Getting top 5 venues

In [55]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CFB Toronto, Downsview East----
                        venue  freq
0                     Airport  0.33
1  Construction & Landscaping  0.33
2                        Park  0.33
3                      Museum  0.00
4                 Pizza Place  0.00


----Design Exchange, Toronto Dominion Centre----
                venue  freq
0         Coffee Shop  0.13
1                Café  0.08
2          Restaurant  0.06
3               Hotel  0.05
4  Italian Restaurant  0.04


----East Toronto----
               venue  freq
0               Park  0.50
1  Convenience Store  0.25
2      Metro Station  0.25
3        Yoga Studio  0.00
4        Music Venue  0.00


----Harbord, University of Toronto----
                 venue  freq
0                 Café  0.14
1           Restaurant  0.06
2  Japanese Restaurant  0.06
3   Italian Restaurant  0.06
4               Bakery  0.06


----Harbourfront East, Toronto Islands, Union Station----
                venue  freq
0         Coffee Shop  0.12
1            

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Construction & Landscaping,Park,Wine Bar,Event Space,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Japanese Restaurant,Gastropub,Bar,Seafood Restaurant
2,East Toronto,Park,Metro Station,Convenience Store,Event Space,Concert Hall,Construction & Landscaping,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
3,"Harbord, University of Toronto",Café,Italian Restaurant,Japanese Restaurant,Restaurant,Bar,Bakery,Bookstore,Dessert Shop,Pub,Yoga Studio
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery


### Clustering neighborhoods

In [59]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3K,North York,"CFB Toronto, Downsview East",M3K,43.737473,-79.464763,1,Airport,Construction & Landscaping,Park,Wine Bar,Event Space,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store
1,M4J,East York,East Toronto,M4J,43.685347,-79.338106,2,Park,Metro Station,Convenience Store,Event Space,Concert Hall,Construction & Landscaping,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
2,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678,3,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop,Gym / Fitness Center,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park
3,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M5J,43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Japanese Restaurant,Gastropub,Bar,Seafood Restaurant


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters